# Baza BASiW
Przypadki i zgony w podziale na:
* Datę
* Województwo
* Powiat
* Wiek
* Płeć
* Stan zaszczepienia
* Choroby współistniejące
* Upośledzenie odporności

## SPRAWDZIĆ: 4. fala od 14.07.2021, ludzie zgłaszają, że są tu złe liczby zgonów!!!

## ŚCIĄGNIJ AKTUALNE DANE RĘCZNIE!!!
<!-- ### Ale obecnie one nie są aktualizowane - tylko do 07.12.2021 -->
https://basiw.mz.gov.pl/index.html#/visualization?id=3761

In [1]:
"""
Set PYTHONPATH for modules in parallel directory

https://stackoverflow.com/questions/3108285/in-python-script-how-do-i-set-pythonpath/3108301

"""

import sys

try:
    sys.path.index('..') # Or os.getcwd() for this directory
except ValueError:
    sys.path.append('..') # Or os.getcwd() for this directory

#--------------------------------------------------------------
import pandas as pd
from share.helper_functions import was_modified_today, display_all
import plotly.express as px
import plotly.graph_objects as go
import os
import datetime
import numpy as np

In [2]:
image_dir = './images'
data_dir = './basiw_data'
data_file_deaths = 'zgony.csv'
data_file_cases = 'zakazenia.csv'
path_deaths = os.sep.join([data_dir,data_file_deaths])
path_cases = os.sep.join([data_dir,data_file_cases])

# if was_modified_today(path_deaths):
#     print('Data up to date')
#     dfd = pd.read_csv(path_deaths, encoding = 'cp1250', sep = ';')
#     dfc = pd.read_csv(path_cases, encoding = 'cp1250', sep = ';')
# else:
#     print('Old data, need to download new data!')

dfd_raw = pd.read_csv(path_deaths, encoding = 'cp1250', sep = ';',low_memory=False)
dfc_raw = pd.read_csv(path_cases, encoding = 'cp1250', sep = ';', low_memory=False)

In [3]:
teryt_file = 'TERC_Urzedowy_2021-12-14.csv'
path_teryt = os.sep.join([data_dir,teryt_file])

# df_teryt = pd.read_csv(path_teryt, encoding = 'cp1250', sep = ';')
df_teryt = pd.read_csv(path_teryt, sep = ';')
df_teryt

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,2,NaN,NaN,NaN,DOLNOŚLĄSKIE,województwo,2021-01-01
1,2,1.0,NaN,NaN,bolesławiecki,powiat,2021-01-01
2,2,1.0,1.0,1.0,Bolesławiec,gmina miejska,2021-01-01
3,2,1.0,2.0,2.0,Bolesławiec,gmina wiejska,2021-01-01
4,2,1.0,3.0,2.0,Gromadka,gmina wiejska,2021-01-01
...,...,...,...,...,...,...,...
4209,32,61.0,1.0,1.0,Koszalin,gmina miejska,2021-01-01
4210,32,62.0,NaN,NaN,Szczecin,miasto na prawach powiatu,2021-01-01
4211,32,62.0,1.0,1.0,Szczecin,gmina miejska,2021-01-01
4212,32,63.0,NaN,NaN,Świnoujście,miasto na prawach powiatu,2021-01-01


In [4]:
df_woj = df_teryt.query('NAZWA_DOD == "województwo"')[['WOJ', 'NAZWA']]
df_woj['NAZWA'] = df_woj['NAZWA'].str.title()

df_woj.index = df_woj.pop('WOJ')
# df_woj
teryt_dict = df_woj.to_dict()['NAZWA']
teryt_dict

{2: 'Dolnośląskie',
 4: 'Kujawsko-Pomorskie',
 6: 'Lubelskie',
 8: 'Lubuskie',
 10: 'Łódzkie',
 12: 'Małopolskie',
 14: 'Mazowieckie',
 16: 'Opolskie',
 18: 'Podkarpackie',
 20: 'Podlaskie',
 22: 'Pomorskie',
 24: 'Śląskie',
 26: 'Świętokrzyskie',
 28: 'Warmińsko-Mazurskie',
 30: 'Wielkopolskie',
 32: 'Zachodniopomorskie'}

In [5]:
dfd_raw

,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony
0,2021-01-02,4,407,K,67.0,65-74,T,NaN,NaN,N,1,N
1,2021-01-02,4,407,K,90.0,85-94,T,NaN,NaN,N,1,N
2,2021-01-02,4,407,M,69.0,65-74,T,NaN,NaN,N,1,N
3,2021-01-02,4,415,M,72.0,65-74,T,NaN,NaN,N,1,N
4,2021-01-02,4,415,M,77.0,75-84,T,NaN,NaN,N,1,N
...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T


## Format data frame

In [6]:
dfd = dfd_raw.copy(deep=True)
dfd['data_rap_zgonu'] = pd.to_datetime(dfd['data_rap_zgonu'], format = "%Y-%m-%d")
dfd['Województwo'] = dfd.teryt_woj
dfd['Województwo'].replace(teryt_dict, inplace=True)
dfd

,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
0,2021-01-02,4,407,K,67.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
1,2021-01-02,4,407,K,90.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
2,2021-01-02,4,407,M,69.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
3,2021-01-02,4,415,M,72.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
4,2021-01-02,4,415,M,77.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


In [7]:
dfd['liczba_zaraportowanych_zgonow'].sum()

59986

In [8]:
dfd_raw['liczba_zaraportowanych_zgonow'].sum()

59986

In [9]:
dfd.shape[0]

58980

Liczba raportowanych zgonów jest większa niż liczba wierszy, bo niektóre liczby raportowanych zgonów są >1

### Czwarta Fala

Przyjmuję początek 4. fali 14.07.2021 – początek wykładniczego wzrostu.

In [10]:
dfd4 = dfd[dfd['data_rap_zgonu'] >= pd.to_datetime(datetime.date(2021, 7, 14))]
# dfd4 = dfd.query('data_rap_zgonu >= "2021-07-14"') $ Tak też jest OK

dfd4

,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
45248,2021-07-14,2,201,K,95.0,95+,N,NaN,NaN,N,1,N,Dolnośląskie
45249,2021-07-14,4,410,K,89.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
45250,2021-07-14,14,1412,K,45.0,45-54,N,NaN,NaN,N,1,N,Mazowieckie
45251,2021-07-14,16,1610,M,65.0,65-74,T,Astra Zeneca,jedna_dawka,N,1,N,Opolskie
45252,2021-07-14,18,1808,M,74.0,65-74,T,NaN,NaN,N,1,N,Podkarpackie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


Test typu wprowadzanej przeze mnie daty:

In [11]:
pd.to_datetime(datetime.date(2021, 7, 14))

Timestamp('2021-07-14 00:00:00')

Test typu daty w tabeli:

In [12]:
dfd4['data_rap_zgonu'][55129]

Timestamp('2021-12-03 00:00:00')

Na dzień 13.12.2021 powinno być 13865:

In [13]:
dfd4['liczba_zaraportowanych_zgonow'].sum()

13865

In [14]:
dfd4['liczba_zaraportowanych_zgonow'].shape[0]

13732

Liczba raportowanych zgonów jest większa niż liczba wierszy, bo niektóre liczby zaraportowanych zgonów w wierszach są większe od 1.

### Procent spośród zgonów na COVID-19 w województwach <br> w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br> (Od 14.07.2021)

In [15]:
dfd4

,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
45248,2021-07-14,2,201,K,95.0,95+,N,NaN,NaN,N,1,N,Dolnośląskie
45249,2021-07-14,4,410,K,89.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
45250,2021-07-14,14,1412,K,45.0,45-54,N,NaN,NaN,N,1,N,Mazowieckie
45251,2021-07-14,16,1610,M,65.0,65-74,T,Astra Zeneca,jedna_dawka,N,1,N,Opolskie
45252,2021-07-14,18,1808,M,74.0,65-74,T,NaN,NaN,N,1,N,Podkarpackie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


In [16]:
df = dfd4.groupby('Województwo').sum()
df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df['Województwo'] = df.index
df


,liczba_zaraportowanych_zgonow,Województwo
Województwo,,
Dolnośląskie,611,Dolnośląskie
Kujawsko-Pomorskie,767,Kujawsko-Pomorskie
Lubelskie,1921,Lubelskie
Lubuskie,250,Lubuskie
Mazowieckie,1992,Mazowieckie
Małopolskie,1121,Małopolskie
Opolskie,435,Opolskie
Podkarpackie,943,Podkarpackie
Podlaskie,872,Podlaskie


In [17]:
Q = f'(w_pelni_zaszczepiony == "N")'

df1 = dfd4.query(Q).groupby('Województwo').sum()
df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df['Zgony nie w pełni zaszczepionych'] = df1

Q = f'(w_pelni_zaszczepiony == "T")'

df1 = dfd4.query(Q).groupby('Województwo').sum()
df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df['Zgony w pełni zaszczepionych'] = df1




df['Zgony nie w pełni zaszczepionych [%]'] = df['Zgony nie w pełni zaszczepionych'] / df['liczba_zaraportowanych_zgonow'] * 100
df['Zgony w pełni zaszczepionych [%]'] = df['Zgony w pełni zaszczepionych'] / df['liczba_zaraportowanych_zgonow'] * 100

df = df.sort_values(['Zgony nie w pełni zaszczepionych [%]'], ascending=False)
df

,liczba_zaraportowanych_zgonow,Województwo,Zgony nie w pełni zaszczepionych,Zgony w pełni zaszczepionych,Zgony nie w pełni zaszczepionych [%],Zgony w pełni zaszczepionych [%]
Województwo,,,,,,
Podkarpackie,943,Podkarpackie,779,164,82.608696,17.391304
Podlaskie,872,Podlaskie,682,190,78.211009,21.788991
Lubelskie,1921,Lubelskie,1498,423,77.980219,22.019781
Łódzkie,911,Łódzkie,693,218,76.070252,23.929748
Świętokrzyskie,461,Świętokrzyskie,342,119,74.186551,25.813449
Dolnośląskie,611,Dolnośląskie,448,163,73.322422,26.677578
Opolskie,435,Opolskie,317,118,72.873563,27.126437
Lubuskie,250,Lubuskie,180,70,72.000000,28.000000
Mazowieckie,1992,Mazowieckie,1421,571,71.335341,28.664659


In [18]:

fig = go.Figure(
    data=[
        go.Bar(
            name = 'Zgony nie w pełni zaszczepionych [%]', 
            x=df['Województwo'], 
            y=df['Zgony nie w pełni zaszczepionych [%]'], 
            offsetgroup=1,
            text = df['Zgony nie w pełni zaszczepionych [%]']
            ),
        go.Bar(
            name = 'Zgony w pełni zaszczepionych [%]', 
            x=df['Województwo'], 
            y=df['Zgony w pełni zaszczepionych [%]'], 
            offsetgroup=2,
            text = df['Zgony w pełni zaszczepionych [%]']
            )
    ]
)

fig.update_traces(
    texttemplate='%{text:2.1f}', 
    textposition='outside',
    )

fig.update_layout(
    barmode='group', 
    width=800, 
    height=500,
    title = "Procent spośród zgonów na COVID-19 w województwach<br>\
    w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(Od 14.07.2021)",
    yaxis_title="Procent spośród zgonów na COVID-19",
    legend_title="",
    uniformtext_minsize=6, 
    uniformtext_mode='show',
    legend=dict(
        y=-0.5,
        xanchor="center",
        x=0.5
        ),
    title_xanchor='center',
    title_x = 0.5,
    title_font_size=16
    )
fig.update_xaxes(tickangle=33)
fig.show()
fig.write_image(os.sep.join([image_dir, 'zgony_szczep_proc_woj_4fala.jpg']), scale=4)


### Procent spośród zgonów na COVID-19 w województwach <br> w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br> (Od 01.01.2021)

In [19]:
df2021 = dfd.groupby('Województwo').sum()
df2021.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df2021['Województwo'] = df2021.index
df2021


,liczba_zaraportowanych_zgonow,Województwo
Województwo,,
Dolnośląskie,3550,Dolnośląskie
Kujawsko-Pomorskie,4013,Kujawsko-Pomorskie
Lubelskie,4588,Lubelskie
Lubuskie,1618,Lubuskie
Mazowieckie,7968,Mazowieckie
Małopolskie,4310,Małopolskie
Opolskie,1430,Opolskie
Podkarpackie,3538,Podkarpackie
Podlaskie,2150,Podlaskie


In [20]:
# df2021['liczba_zaraportowanych_zgonow'].sum()

In [21]:
Q = f'(w_pelni_zaszczepiony == "N")'

df1_2021 = dfd.query(Q).groupby('Województwo').sum()
df1_2021.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df2021['Zgony nie w pełni zaszczepionych'] = df1_2021

Q = f'(w_pelni_zaszczepiony == "T")'

df1_2021 = dfd.query(Q).groupby('Województwo').sum()
df1_2021.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df2021['Zgony w pełni zaszczepionych'] = df1_2021




df2021['Zgony nie w pełni zaszczepionych [%]'] = df2021['Zgony nie w pełni zaszczepionych'] / df2021['liczba_zaraportowanych_zgonow'] * 100
df2021['Zgony w pełni zaszczepionych [%]'] = df2021['Zgony w pełni zaszczepionych'] / df2021['liczba_zaraportowanych_zgonow'] * 100

df2021 = df2021.sort_values(['Zgony nie w pełni zaszczepionych [%]'], ascending=False)
df2021

,liczba_zaraportowanych_zgonow,Województwo,Zgony nie w pełni zaszczepionych,Zgony w pełni zaszczepionych,Zgony nie w pełni zaszczepionych [%],Zgony w pełni zaszczepionych [%]
Województwo,,,,,,
Lubuskie,1618,Lubuskie,1534,84,94.808405,5.191595
Podkarpackie,3538,Podkarpackie,3343,195,94.488412,5.511588
Wielkopolskie,5181,Wielkopolskie,4864,317,93.881490,6.118510
Dolnośląskie,3550,Dolnośląskie,3328,222,93.746479,6.253521
Pomorskie,3425,Pomorskie,3207,218,93.635036,6.364964
Łódzkie,4135,Łódzkie,3864,271,93.446191,6.553809
Świętokrzyskie,2108,Świętokrzyskie,1969,139,93.406072,6.593928
Śląskie,7218,Śląskie,6736,482,93.322250,6.677750
Kujawsko-Pomorskie,4013,Kujawsko-Pomorskie,3723,290,92.773486,7.226514


In [22]:

fig = go.Figure(
    data=[
        go.Bar(
            name = 'Zgony nie w pełni zaszczepionych [%]', 
            x=df2021['Województwo'], 
            y=df2021['Zgony nie w pełni zaszczepionych [%]'], 
            offsetgroup=1,
            text = df2021['Zgony nie w pełni zaszczepionych [%]']
            ),
        go.Bar(
            name = 'Zgony w pełni zaszczepionych [%]', 
            x=df2021['Województwo'], 
            y=df2021['Zgony w pełni zaszczepionych [%]'], 
            offsetgroup=2,
            text = df2021['Zgony w pełni zaszczepionych [%]']
            )
    ]
)

fig.update_traces(
    texttemplate='%{text:2.1f}', 
    textposition='outside',
    )

fig.update_layout(
    barmode='group', 
    width=800, 
    height=500,
    title = "Procent spośród zgonów na COVID-19 w województwach<br>\
    w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(Od 01.07.2021)",
    yaxis_title="Procent spośród zgonów na COVID-19",
    legend_title="",
    uniformtext_minsize=8, 
    uniformtext_mode='show',
    legend=dict(
        y=-0.5,
        xanchor="center",
        x=0.5
        ),
    title_xanchor='center',
    title_x = 0.5,
    title_font_size=16
    )
fig.update_xaxes(tickangle=33)
fig.show()
fig.write_image(os.sep.join([image_dir, 'zgony_szczep_proc_woj_2021.jpg']), scale=4)


### Liczba zgonów na COVID-19 w województwach <br> w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br> (Od 14.07.2021)

In [23]:
df = dfd4.groupby('Województwo').sum()
df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df['Województwo'] = df.index
df


,liczba_zaraportowanych_zgonow,Województwo
Województwo,,
Dolnośląskie,611,Dolnośląskie
Kujawsko-Pomorskie,767,Kujawsko-Pomorskie
Lubelskie,1921,Lubelskie
Lubuskie,250,Lubuskie
Mazowieckie,1992,Mazowieckie
Małopolskie,1121,Małopolskie
Opolskie,435,Opolskie
Podkarpackie,943,Podkarpackie
Podlaskie,872,Podlaskie


In [24]:
Q = f'(w_pelni_zaszczepiony == "N")'

df1 = dfd4.query(Q).groupby('Województwo').sum()
df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df['Zgony nie w pełni zaszczepionych'] = df1

Q = f'(w_pelni_zaszczepiony == "T")'

df1 = dfd4.query(Q).groupby('Województwo').sum()
df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
df['Zgony w pełni zaszczepionych'] = df1




df['Zgony nie w pełni zaszczepionych [%]'] = df['Zgony nie w pełni zaszczepionych'] / df['liczba_zaraportowanych_zgonow'] * 100
df['Zgony w pełni zaszczepionych [%]'] = df['Zgony w pełni zaszczepionych'] / df['liczba_zaraportowanych_zgonow'] * 100

df = df.sort_values(['Zgony nie w pełni zaszczepionych'], ascending=False)
df

,liczba_zaraportowanych_zgonow,Województwo,Zgony nie w pełni zaszczepionych,Zgony w pełni zaszczepionych,Zgony nie w pełni zaszczepionych [%],Zgony w pełni zaszczepionych [%]
Województwo,,,,,,
Lubelskie,1921,Lubelskie,1498,423,77.980219,22.019781
Mazowieckie,1992,Mazowieckie,1421,571,71.335341,28.664659
Śląskie,1232,Śląskie,834,398,67.694805,32.305195
Małopolskie,1121,Małopolskie,799,322,71.275647,28.724353
Podkarpackie,943,Podkarpackie,779,164,82.608696,17.391304
Łódzkie,911,Łódzkie,693,218,76.070252,23.929748
Podlaskie,872,Podlaskie,682,190,78.211009,21.788991
Wielkopolskie,790,Wielkopolskie,549,241,69.493671,30.506329
Kujawsko-Pomorskie,767,Kujawsko-Pomorskie,525,242,68.448501,31.551499


In [25]:

fig = go.Figure(
    data=[
        go.Bar(
            name = 'Zgony nie w pełni zaszczepionych', 
            x=df['Województwo'], 
            y=df['Zgony nie w pełni zaszczepionych'], 
            offsetgroup=1,
            text = df['Zgony nie w pełni zaszczepionych']
            ),
        go.Bar(
            name = 'Zgony w pełni zaszczepionych', 
            x=df['Województwo'], 
            y=df['Zgony w pełni zaszczepionych'], 
            offsetgroup=2,
            text = df['Zgony w pełni zaszczepionych']
            )
    ]
)

fig.update_traces(
    texttemplate='%{text:2.1f}', 
    textposition='outside',
    )

fig.update_layout(
    barmode='group', 
    width=800, 
    height=500,
    title = "Liczba zgonów na COVID-19 w województwach<br>\
    w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(Od 14.07.2021)",
    yaxis_title="Liczba zgonów",
    legend_title="",
    uniformtext_minsize=6, 
    uniformtext_mode='show',
    legend=dict(
        y=-0.5,
        xanchor="center",
        x=0.5
        ),
    title_xanchor='center',
    title_x = 0.5,
    title_font_size=16
    )
fig.update_xaxes(tickangle=33)
fig.show()
fig.write_image(os.sep.join([image_dir, 'zgony_szczep_proc_woj_4fala.jpg']), scale=4)


### Liczba zgonów na COVID-19 w województwach na 100 tys. mieszkańców<br> w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br> (Od 14.07.2021)

In [26]:
dfpopwoj = pd.read_csv('/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/GUS/gus_data/wojewodztwa_ludnosc_2021_GUS.csv')

In [27]:
dfpopwoj

,TERYT,Województwo,Ludność
0,2,Dolnośląskie,2891321
1,4,Kujawsko-pomorskie,2061942
2,6,Lubelskie,2095258
3,8,Lubuskie,1007145
4,10,Łódzkie,2437970
5,12,Małopolskie,3410441
6,14,Mazowieckie,5425028
7,16,Opolskie,976774
8,18,Podkarpackie,2121229
9,20,Podlaskie,1173286


In [28]:
dfpopwoj = pd.read_csv('/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/GUS/gus_data/wojewodztwa_ludnosc_2021_GUS.csv')
dfpopwoj['Województwo'] = dfpopwoj['Województwo'].str.title()
dfpopwoj.drop(['TERYT'], axis='columns', inplace=True)
dfpopwoj.index = dfpopwoj['Województwo']
dfpopwoj.drop(['Województwo'], axis='columns' , inplace = True)
pop_dict = dfpopwoj.to_dict()['Ludność']
pop_dict

{'Dolnośląskie': 2891321,
 'Kujawsko-Pomorskie': 2061942,
 'Lubelskie': 2095258,
 'Lubuskie': 1007145,
 'Łódzkie': 2437970,
 'Małopolskie': 3410441,
 'Mazowieckie': 5425028,
 'Opolskie': 976774,
 'Podkarpackie': 2121229,
 'Podlaskie': 1173286,
 'Pomorskie': 2346671,
 'Śląskie': 4492330,
 'Świętokrzyskie': 1224626,
 'Warmińsko-Mazurskie': 1416495,
 'Wielkopolskie': 3496450,
 'Zachodniopomorskie': 1688047}

In [29]:
 def prepare_df(deaths_df: pd.DataFrame, from_date : datetime.date) -> pd.DataFrame:
    base_df = deaths_df[deaths_df['data_rap_zgonu'] >= pd.to_datetime(from_date)]

    df = base_df.groupby('Województwo').sum()
    df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Województwo'] = df.index
    df['Ludność'] = df['Województwo'].replace(pop_dict)
    # df

    Q = f'(w_pelni_zaszczepiony == "N")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Zgony nie w pełni zaszczepionych'] = df1

    Q = f'(w_pelni_zaszczepiony == "T")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Zgony w pełni zaszczepionych'] = df1




    df['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony nie w pełni zaszczepionych'] / df['Ludność'] * 1e5
    df['Zgony w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony w pełni zaszczepionych'] / df['Ludność'] * 1e5

    df = df.sort_values(['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'], ascending=False)
    return df

In [30]:
def plot_df(from_date : datetime.date, df : pd.DataFrame) -> pd.DataFrame:
    nie = 'Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'
    tak = 'Zgony w pełni zaszczepionych na 100 tys. mieszkańców'

    fig = go.Figure(
        data=[
            go.Bar(
                name = nie, 
                x=df['Województwo'], 
                y=df[nie], 
                offsetgroup=1,
                text = df[nie]
                ),
            go.Bar(
                name = tak, 
                x=df['Województwo'], 
                y=df[tak], 
                offsetgroup=2,
                text = df[tak]
                )
        ]
    )

    fig.update_traces(
        texttemplate='%{text:2.1f}', 
        textposition='outside',
        )

    fig.update_layout(
        barmode='group', 
        width=800, 
        height=500,
        title = "Liczba zgonów na COVID-19 w województwach na 100 tys. mieszkańców<br>\
        w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(Od " + str(from_date) + ")",
        yaxis_title="Liczba zgonów na 100 tys.",
        legend_title="",
        uniformtext_minsize=6, 
        uniformtext_mode='show',
        legend=dict(
            y=-0.5,
            xanchor="center",
            x=0.5
            ),
        title_xanchor='center',
        title_x = 0.5,
        title_font_size=16
        )
    fig.update_xaxes(tickangle=33)
    fig.show()
    fig.write_image(os.sep.join([image_dir, 'zgony_szczep_1e5_woj_od'+str(from_date)+'.jpg']), scale=4)
    return


In [31]:
from_date = datetime.date(2021,7,14)

df = prepare_df(dfd, from_date)
plot_df(from_date, df)

In [32]:
from_date = datetime.date(2021,1,1)

df = prepare_df(dfd, from_date)
plot_df(from_date, df)

In [62]:
dfd.groupby(['dawka_ost']).sum()

,teryt_woj,teryt_pow,wiek,liczba_zaraportowanych_zgonow
dawka_ost,,,,
jedna_dawka,55094,5612188,252493.0,3269
pelna_dawka,69738,7105723,332369.0,4304
przypominajaca,1282,131001,7160.0,88
uzupełniająca,496,50672,2561.0,36


In [91]:
import numpy as np
pd.DataFrame(dfd['dawka_ost'].to_numpy())

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
58975,NaN
58976,pelna_dawka
58977,pelna_dawka
58978,pelna_dawka


In [101]:
type(dfd['dawka_ost'][0])

float

In [114]:
dfd_test = dfd.copy(deep=True)

In [116]:
dfd_test['dawka_ost'] = dfd_test['dawka_ost'].to_string()
dfd_test

In [112]:
dfd[dfd['dawka_ost'] == 'nan']

,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo


In [ ]:
 def prepare_df1(deaths_df: pd.DataFrame, from_date : datetime.date) -> pd.DataFrame:
    base_df = deaths_df[deaths_df['data_rap_zgonu'] >= pd.to_datetime(from_date)]

    df = base_df.groupby('Województwo').sum()
    df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Województwo'] = df.index
    df['Ludność'] = df['Województwo'].replace(pop_dict)
    # df

    Q = f'(dawka_ost == "jedna_dawka")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Jedna dawka'] = df1

    Q = f'(dawka_ost == "pelna_dawka")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Pełna dawka'] = df1

    Q = f'(dawka_ost == "przypominajaca")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Przypominająca'] = df1

    Q = f'(dawka_ost == "uzupełniająca")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Uzupełniająca'] = df1

    df['Zgony po jednej dawce, na 100 tys. mieszkańców'] = df['Jedna dawka'] / df['Ludność'] * 1e5
    df['Zgony po pełnej dawce, na 100 tys. mieszkańców'] = df['Pełna dawka'] / df['Ludność'] * 1e5
    df['Zgony po dawce przypominającej, na 100 tys. mieszkańców'] = df['Przypominająca'] / df['Ludność'] * 1e5
    df['Zgony po dawce uzupełniającej, na 100 tys. mieszkańców'] = df['Uzupełniająca'] / df['Ludność'] * 1e5

    df = df.sort_values(['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'], ascending=False)
    return df